# Conservation-based Synthetic Lethal Search

## Introduction

### Rationale

### Use-cases:
* Prioritize human candidate synthetic lethal interactions based on prior evidence of interaction in yeast SL screens
* _de novo_ discovery of SL interactions

### Approach
This notebook re-implements the approach outlined in Srivas et al. (2016)

### Usage:
Add genes of interest to "inputGenes" value, then run the next step.
Example: inputGenes = "'DDX3X','DICER1','DROSHA','TNFRSF14','TRAF7','TSC1','POLG','FBXO11','PRDM1','RFWD3','AMER1','LZTR1','ATP2B3'"

### Workflow Overview

### Datasets
#### Yeast Synthetic Lethal Interactions
Constanzo et al. (2016)
#### Human to Yeast Ortholog Mapping
detailed treatement in the accompanying notebook (Mapping human to yeast orthologs)
#### Human Tumor Suppressor Genes


### References
* Costanzo M, VanderSluis B, Koch EN, Baryshnikova A, Pons C, Tan G, Wang W, Usaj M, Hanchard J, Lee SD, Pelechano V, Styles EB, Billmann M, van Leeuwen J, van Dyk N, Lin ZY, Kuzmin E, Nelson J, Piotrowski JS, Srikumar T, Bahr S, Chen Y, Deshpande R, Kurat CF, Li SC, Li Z, Usaj MM, Okada H, Pascoe N, San Luis BJ, Sharifpoor S, Shuteriqi E, Simpkins SW, Snider J, Suresh HG, Tan Y, Zhu H, Malod-Dognin N, Janjic V, Przulj N, Troyanskaya OG, Stagljar I, Xia T, Ohya Y, Gingras AC, Raught B, Boutros M, Steinmetz LM, Moore CL, Rosebrock AP, Caudy AA, Myers CL, Andrews B, Boone C. **A global genetic interaction network maps a wiring diagram of cellular function.** Science. 2016 Sep 23;353(6306). pii: aaf1420. PubMed PMID: 27708008; PubMed Central PMCID: PMC5661885.
* Srivas R, Shen JP, Yang CC, Sun SM, Li J, Gross AM, Jensen J, Licon K, Bojorquez-Gomez A, Klepper K, Huang J, Pekin D, Xu JL, Yeerna H, Sivaganesh V, Kollenstart L, van Attikum H, Aza-Blanc P, Sobol RW, Ideker T. **A Network of Conserved Synthetic Lethal Interactions for Exploration of Precision Cancer Therapy**. Mol Cell. 2016 Aug 4;63(3):514-25. doi:10.1016/j.molcel.2016.06.022.Epub 2016 Jul 21. PubMed PMID: 27453043; PubMed Central PMCID: PMC5209245. 

## Preamble
This section describes how to setup the analysis environment appropriately, including google cloud platform authentication and importing all the relevant python libraries.

### Setup Analysis Environment

In [ ]:
#! pip3 install google-cloud-bigquery

In [ ]:
# google cloud authentication
from google.cloud import bigquery


In [ ]:
# import modules
import sys
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import stats 
import numpy as np
import json
import statsmodels.stats.multitest as multi
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
import plotly
import plotly.express as px
import pyarrow


In [ ]:
# users need to run the following commend in their local machine or throught the notebook.
# Make sure to install the google cloud in the local envirionment. For more detail of gcloud installation, please see support from https://cloud.google.com/sdk/docs/install

!gcloud auth application-default login

In [ ]:
# Choose the project to be used for bigquery
project_id='syntheticlethality'
client = bigquery.Client(project_id) # Replace XXXXXXXX with your project ID

In [ ]:
%load_ext google.cloud.bigquery

## Define a set of cancer-relevant tumor suppressor genes (TSGs)

In this workflow, the search for relevant synthetic lethal interactions is seeded by defining a set of tumor suppressor genes (TSGs) of interest. There are various strategies for obtaining such a list, here we give an example of mining the [COSMIC Cancer Gene Census](https://cancer.sanger.ac.uk/census) for TSG annotations and then prioritizing the list based on driver status or frequency of alteration in a cancer type of interest.

If you want to get the SL interactions for genes of interest, please add the genes to "inputGenes".

In [ ]:
query = ''' 
            SELECT *
            FROM `isb-cgc.COSMIC_v90_grch38.Cancer_Gene_Census` 
            WHERE Role_in_Cancer = "TSG"
        '''
tsg = client.query(query).result().to_dataframe()

In [ ]:
tsg.head()

In [ ]:
# generate a list for inputGenes (Please go to the next block if you want to use your genes instead of tumor suppressor genes)
tumor_suppressor_genes = tsg["Gene_Symbol"].tolist() 
inputGenes = ["'"+x+"'" for x in tumor_suppressor_genes]
inputGenes = ','.join(inputGenes)
inputGenes

In [ ]:
# please skip this block if you want to keep using tumor suppressor genes as an input
#inputGenes = ""

## Map Yeast Orthologs & Get SL insteractions

In [ ]:
sql = '''
WITH
--- Retreive YeastSymbols mapped to HumanSymbols for the input genes
INPUT_H2Y AS (
  SELECT YeastSymbol
    FROM `syntheticlethality.gene_information.human2Yeast`
   WHERE HumanSymbol IN (__INPUTGENES__) AND
         AlgorithmsMatch >= __ALGORITHMCUTOFF__
),

--- Identify protein-protein interactions using the YeastSymbols (left match)
Yeast_ITX1 AS (
  SELECT UPPER(Query_allele_name)       AS Interactor1, 
         UPPER(Array_allele_name)       AS Interactor2,
         Genetic_interaction_score_____ AS Interaction_score,
         P_value
    FROM `syntheticlethality.CellMap.CellMap`
   WHERE (Genetic_interaction_score_____ < __SCORECUTOFF__ AND P_value < __PvalueCUTOFF__) AND
         (UPPER(Query_allele_name) IN (SELECT YeastSymbol FROM INPUT_H2Y))
   
),

--- Identify protein-protein interactions using the YeastSymbols (right match)
Yeast_ITX2 AS (
  SELECT UPPER(Array_allele_name)       AS Interactor1, 
         UPPER(Query_allele_name)       AS Interactor2,
         Genetic_interaction_score_____ AS Interaction_score,
         P_value
    FROM `syntheticlethality.CellMap.CellMap`
   WHERE (Genetic_interaction_score_____ < __SCORECUTOFF__ AND P_value < __PvalueCUTOFF__) AND
         (UPPER(Array_allele_name) IN (SELECT YeastSymbol FROM INPUT_H2Y))
   
),

--- Union interaction tables
Union_ITX AS (
  SELECT * FROM Yeast_ITX1
   UNION ALL
  SELECT * FROM Yeast_ITX2
)

--- Convert YeastSymbols to HumanSymbols in the protein-protein interations
SELECT DISTINCT 
       GINFO1.EntrezID        AS EntrezID_Input,
       H2Y1.HumanSymbol       AS Gene_Input,
---       Add if you want to know what yeast genes are involved
---       YITX.Interactor1       AS Gene_Input_Yeast,
       GINFO2.EntrezID        AS EntrezID_SL_Candidate,
       H2Y2.HumanSymbol       AS Gene_SL_Candidate,
---       Add if you want to know what yeast genes are involved
---       YITX.Interactor2       AS Gene_SL_Candidate_Yeast,
       YITX.Interaction_score AS Interaction_score,
       YITX.P_value           AS P_value
       
  FROM Union_ITX AS YITX
       LEFT JOIN `syntheticlethality.gene_information.human2Yeast`                       AS H2Y1   ON YITX.Interactor1 = H2Y1.YeastSymbol
       LEFT JOIN `syntheticlethality.gene_information.human2Yeast`                       AS H2Y2   ON YITX.Interactor2 = H2Y2.YeastSymbol
       LEFT JOIN `syntheticlethality.gene_information.gene_info_human_HGNC` AS GINFO1 ON H2Y1.HumanID = GINFO1.HGNCID
       LEFT JOIN `syntheticlethality.gene_information.gene_info_human_HGNC` AS GINFO2 ON H2Y2.HumanID = GINFO2.HGNCID
       
 WHERE (H2Y1.HumanSymbol IS NOT NULL AND YITX.Interactor1 IS NOT NULL) AND
       (H2Y2.HumanSymbol IS NOT NULL AND YITX.Interactor2 IS NOT NULL)

'''
# select the thresholds to be used
cutoff_algorithmMatchNo = "3"
cutoff_score = "-0.35"
cutoff_p = "0.01"

sql = sql.replace("__INPUTGENES__", inputGenes)
sql = sql.replace("__ALGORITHMCUTOFF__", cutoff_algorithmMatchNo)
sql = sql.replace("__SCORECUTOFF__", cutoff_score)
sql = sql.replace("__PvalueCUTOFF__", cutoff_p)

res = client.query(sql).to_dataframe()



## Get Yeast SL Interactions

In [ ]:
# shore the SL partner genes for the input genes
res


## Write to file & bigQuery Table

In [ ]:
res.to_csv(path_or_buf='conserved_SL_output.csv', index=False)

# END